Data Mining 作業一 Association Rules Frequent Itemset

# Description

此次作業主要目的在讓同學學習運用 python 由 Foodmart Database 超級市場的顧客及其交易資料中，

以 **Apriori**, **FP-Tree** Algorithm 探勘 *Frequent Itemsets*, *Association Rules*, *Multi-level Association Rules*, *Quantitative Association Rules*。

- 作業給定的 Foodmart Database 已經附在 WM5 平台上。
- 作業每人繳交一份報告，檔案類型以 pdf 為限。上傳檔名格式為 學號_HW1，EX:
110753XXX_HW1.pdf
- 此次作業交易資料只針對 1998 年的資料 (`sales_fact_1998` + `sales_dec_1998`)
- 此次作業可以使用現有套件執行運算。

In [ ]:
import pandas as pd
import numpy as np
from os import listdir

In [ ]:
# working directory on google drive
#wdir = "drive/MyDrive/NCCU_courses/資料挖掘/hw1/"

In [ ]:
# working directory on local
wdir = "./"

In [ ]:
# list directory in working directory
listdir(wdir)

In [ ]:
# reading .csv
sales_fact_1998 = pd.read_csv(wdir + "hw1_data/sales_fact_1998.csv")
sales_dec_1998 = pd.read_csv(wdir + "hw1_data/sales_fact_dec_1998.csv")

# Concat two df

concat `sales_fact_1998` & `sales_dec_1998` into one dataframe

In [ ]:
# concat and reindex
df = pd.concat([sales_fact_1998, sales_dec_1998], ignore_index=True)

In [ ]:
# make sure we had the concatenation right
assert len(sales_fact_1998) + len(sales_dec_1998) == df.shape[0]

In [ ]:
# take a peek at the dataframe
print("shape:", df.shape)
df.sample(3)

# Pre-processing

In [ ]:
# check if there is any missing value and their data types
df.info()

In [ ]:
# remove "NT$" then convert store_sales and store_cost to float
df["store_sales"] = [float(sale[3:]) if type(sale) == str else sale for sale in df["store_sales"]]
df["store_cost"] = [float(sale[3:]) if type(sale) == str else sale for sale in df["store_cost"]]

In [ ]:
df["store_sales"].dtype, df["store_cost"].dtype

In [ ]:
# drop store_sales, store_cost, unit_sales columns
to_use = ["customer_id", "time_id", "store_id", "promotion_id", "product_id"]
df_done = df[to_use]
#df_done.sample()

## map out `product_id` from `product.csv`

In [ ]:
df_product = pd.read_csv(wdir + "hw1_data/product.csv")
df_product.sample(3)

In [ ]:
df_product["product_id"].unique()

In [ ]:
df_done["product_id"].unique()

In [ ]:
df_product[["product_id", "product_name"]]

In [ ]:
id_name = {
    p_id: p_name
    for p_id, p_name in zip(df_product["product_id"], df_product["product_name"])}

In [ ]:
id_name

In [ ]:
df_done["product_name"] = [id_name[p_id] for p_id in df_done["product_id"]]

In [ ]:
df_done

## Input preparation

### way 2 Groupby ["customer_id", "time_id", "store_id", "promotion_id"]

can tell product class from `product_*.csv`

In [ ]:
_df_groupby = df_done.sort_values(to_use)

In [ ]:
groupby_dict = {}
for c, t, s, p, p_id, product_name in _df_groupby[:].values:
    #print(c, t, s, p, product)
    if (c, t, s, p) not in groupby_dict:
        groupby_dict[(c, t, s, p)] = ()
        _temp = groupby_dict[(c, t, s, p)] + (product_name, )
        groupby_dict[(c, t, s, p)] = _temp
    elif (c, t, s, p) in groupby_dict:
        _temp = groupby_dict[(c, t, s, p)] + (product_name, )
        groupby_dict[(c, t, s, p)] = _temp
        
groupby_dict

In [ ]:
transactions = [items for items in groupby_dict.values()]

# Question 1
請利用 **Apriori** 演算法，從 Foodmart 資料庫的交易資料中，探勘符合 `Minimum Support = 0.0001` 且 `Minimum Confidence = 0.9` 的 **Association Rules**，並列出 Confidence 最高的前 10 條 Rules 以及 lift 最高的前 10 條，並比較這兩者的異同。若無法跑出結果，請簡述其原因。

using fastest apriori algorithm from https://efficient-apriori.readthedocs.io/en/latest/?badge=latest

others apriori algorithms:
- https://github.com/ymoch/apyori
    - https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/
    - https://ubiops.com/how-to-build-and-implement-a-recommendation-system-from-scratch-in-python/
- https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [ ]:
from efficient_apriori import apriori

In [ ]:
%%time
itemsets, rules = apriori(transactions, min_support=0.0001, min_confidence=0.9)

In [ ]:
print(f"having {len(itemsets)} itemsets.")

In [ ]:
rules

## 10 highest confidence

In [ ]:
confidences = [rule.confidence for rule in rules]
print(min(confidences), max(confidences))

In [ ]:
for rule in rules:
    print(rule.confidence, rule)
    print()

## 10 highest lift

In [ ]:
lifts = [rule.lift for rule in rules]
print(min(lifts), max(lifts))

In [ ]:
for rule in rules:
    if rule.lift > 9000:
        print(rule.lift, rule)
        print()

## difference between 10 highest confidence and 10 highest lift

# Question 2
請利用 **FP-Growth** 演算法，從 Foodmart 資料庫的交易資料中，探勘符合 `Minimum Support = 0.0001` 且 `Minimum Confidence = 0.9` 的 **Association Rules**，並列出 Confidence 最高的前 10 條 Rules 以及 lift 最高的前 10 條，並比較這兩者的異同。若無法跑出結果，請簡述其原因。

FP-Growth algorithm from: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/
- https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#metrics

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [ ]:
dataset = [list(data) for data in groupby_dict.values()]
#dataset

In [ ]:
te = TransactionEncoder()
te_array = te.fit_transform(dataset)
df_te = pd.DataFrame(te_array, columns=te.columns_)
#df_te

In [ ]:
fq_itemsets = fpgrowth(df_te, min_support=0.0001, use_colnames=True)
#fq_itemsets

In [ ]:
rules = association_rules(fq_itemsets, metric="confidence", min_threshold=0.9)
rules

## 10 highest confidence

In [ ]:
rules.sort_values(["confidence"])[::-1][:10][["antecedents", "consequents"]].values

## 10 highest lift

In [ ]:
rules.sort_values(["lift"])[::-1][:10][["antecedents", "consequents"]].values

## difference between 10 highest confidence and 10 highest lift

# Question 3
有時候我們有興趣的資料不只有產品間的資訊,也會想要由 User Profile 探勘顧客的基本資料。

在給定 `Minimum Support = 0.05` 且 `Minimum Confidence = 0.9` 的條件下, 探勘 Foodmart 顧客基本資料的屬性 `{State_Province, Yearly_Income, Gender, Total_Children, Num_Children_at_Home, Education, Occupation, Houseowner, Num_cars,owned}` 間的 **association rule**。(列出 10 條)

library: https://github.com/firefly-cpp/NiaARM
- https://niaarm.readthedocs.io/en/latest/getting_started.html

In [ ]:
from niaarm import Dataset

In [ ]:
df_customers = pd.read_csv(f"{wdir}/hw1_data/customer.csv")
df_customers.sample(3)

In [ ]:
df_customers = df_customers[[
    "state_province", "yearly_income", "gender", 
    "total_children", "num_children_at_home", "education", 
    "occupation", "houseowner", "num_cars_owned"]]

In [ ]:
df_customers.sample(3)

In [ ]:
dataset = Dataset(df_customers)
#print(dataset)

In [ ]:
from niapy.algorithms.basic import DifferentialEvolution
from niaarm import get_rules

In [ ]:
algorithm = DifferentialEvolution(
    population_size=30, 
    differential_weight=0.8, 
    crossover_probability=0.9)

#metrics = ('support', 'confidence', 'inclusion', 'amplitude')
rules, run_time = get_rules(
    dataset, algorithm, {"support": 0.0001, "confidence": 0.9}, max_iters=50, logging=False)
rules.to_csv(wdir + 'niaarm_rules.csv')

In [ ]:
# read `niaarm_rules.csv`
df_niaarm_rules = pd.read_csv(wdir + "niaarm_rules.csv")
df_niaarm_rules = df_niaarm_rules[["antecedent", "consequent", "fitness", "support", "confidence", "lift"]]
df_niaarm_rules_sorted = df_niaarm_rules.sort_values(["lift", "support", "confidence"])[::-1]
for i, rule in enumerate(df_niaarm_rules_sorted[["antecedent", "consequent"]][:10].values):
    print(f"rule{i+1}:")
    print(rule)
    print()

# Question 4
請探勘 Foodmart 資料庫中,顧客背景資料與其交易資料之間的關係 (Quantitative Association Rules)。例如 80% 女性顧客常買保養品。請自行嘗試設定 `Minimum Support` `Minimum Confidence`, 找出 10 條你覺得有意義的 Rules。請說明你的作法及相關參數設定。

In [ ]:
sales_fact_1998 = pd.read_csv(wdir + "hw1_data/sales_fact_1998.csv")
sales_dec_1998 = pd.read_csv(wdir + "hw1_data/sales_fact_dec_1998.csv")

In [ ]:
df_customer = pd.read_csv(wdir + "hw1_data/customer.csv")
df_sales = pd.concat([sales_fact_1998, sales_dec_1998], ignore_index=True)
print(df_sales.shape, df_customer.shape)

In [ ]:
#print(df_customer.columns)
df_customer.sample(3)

In [ ]:
# if gonna map sales product_id
df_product = pd.read_csv(wdir + "hw1_data/product.csv")

id_name = {
    p_id: p_name
    for p_id, p_name in zip(df_product["product_id"], df_product["product_name"])}

df_sales["product_name"] = [id_name[p_id] for p_id in df_sales["product_id"]]
df_sales.sample(3)

some ideas:
- groupby `customer_id` -> product_name
    - map ('birthdate', 'marital_status', 'yearly_income', 'gender', 'total_children', 'num_children_at_home', 'education', 'occupation', 'houseowner', 'num_cars_owned')
- groupby `time_id` -> product_name
    - map ('the_month')

In [ ]:
df_customer = df_customer[[
    "customer_id", "state_province", "yearly_income", "gender", 
    "total_children", "num_children_at_home", "education", 
    "occupation", "houseowner", "num_cars_owned"]]
df_customer.sample(3)

In [ ]:
df_join = pd.merge(df_sales, df_customer, on=["customer_id"])
print(df_join.shape)
df_join.sample(3)

In [ ]:
df_join = df_join[[
    "product_name", "yearly_income", 
    "gender", "total_children", "education", 
    "occupation", "houseowner", "num_cars_owned"]]

In [ ]:
#df_join["promotion_id"] = [1 if p != 0 else 0 for p in df_join["promotion_id"]]

In [ ]:
df_join

In [ ]:
from niaarm import Dataset
from niapy.algorithms.basic import DifferentialEvolution
from niaarm import get_rules

In [ ]:
dataset = Dataset(df_join)

In [ ]:
algorithm = DifferentialEvolution(
    population_size=30, 
    differential_weight=0.8, 
    crossover_probability=0.9)

rules, run_time = get_rules(
    dataset, algorithm, {"support": 0.8, "confidence": 0.9}, max_iters=50, logging=True)
print(len(rules))
rules.to_csv(wdir + 'niaarm_rules_q4.csv')

In [ ]:
# read `niaarm_rules.csv`
df_niaarm_rules = pd.read_csv(wdir + "niaarm_rules_q4.csv")
df_niaarm_rules = df_niaarm_rules[["antecedent", "consequent", "fitness", "support", "confidence", "lift"]]
df_niaarm_rules_sorted = df_niaarm_rules.sort_values(["confidence"])[::-1]
for i, rule in enumerate(df_niaarm_rules_sorted[["antecedent", "consequent", "confidence"]][:20].values):
    #print(f"rule{i+1}:")
    print(rule)
    print()

# Question 5
在美國由於聖誕節，12 月是購物的旺季。請探勘分析比較 12 月與 1 ~ 11 月的顧客購物行為。
有哪些相似的地方，有哪些差異的地方？

some ideas:
- groupby `time_id` -> # of people buying
    - map ('the_month')
- groupby `time_id`, `product_id` -> # of total unit_sales
    - map ('the_month'), ('product_class_id')

In [ ]:
df_time = pd.read_csv(wdir + "hw1_data/time_by_day.csv")
#df_time.sample(3)

In [ ]:
df_time_12 = df_time[["time_id", "month_of_year"]].loc[df_time["month_of_year"] == 12]

In [ ]:
df_time_1_11 = df_time[["time_id", "month_of_year"]].loc[df_time["month_of_year"] != 12]
df_time_1_11

In [ ]:
df_sales.sample(3)

In [ ]:
month = []
not_d_id = [i for i in df_time_1_11["time_id"].values]
for t_id in df_sales["time_id"]:
    if t_id in not_d_id:
        month.append("not_december")
    else:
        month.append("december")
#month

In [ ]:
'december' in month, "not_december" in month

In [ ]:
df_sales["month"] = month

In [ ]:
len(df_sales.loc[df_sales["month"] == "december"])

In [ ]:
print(len(df_sales.loc[df_sales["month"] == "not_december"]))
len(df_sales.loc[df_sales["month"] != "december"]) / 11

In [ ]:
df_sales

In [ ]:
len(df_sales.loc[df_sales["month"] == "december"].loc[df_sales["promotion_id"] == 0])

In [ ]:
len(df_sales.loc[df_sales["month"] == "december"].loc[df_sales["promotion_id"] != 0])

In [ ]:
len(df_sales.loc[df_sales["month"] == "not_december"].loc[df_sales["promotion_id"] == 0])

In [ ]:
len(df_sales.loc[df_sales["month"] == "not_december"].loc[df_sales["promotion_id"] != 0])

In [ ]:
3278 / (3278 + 15047)

In [ ]:
44292 / (44292 + 120266)